函数作参数

### 函数作参数

In [2]:
add(x,y) = x + y
sqrt(add(3,6)) |> display
(sqrt ∘ add)(3, 6) |> display

3.0

3.0

In [4]:
multiply2(x) = x*2
map(multiply2, [1, 2, 3])

3-element Vector{Int64}:
 2
 4
 6

In [8]:
map(+, [1, 2, 3], [10, 20, 30, 400, 5000])

3-element Vector{Int64}:
 11
 22
 33

### 匿名函数作参数

如julia自带的map函数: ***julia\base\abstractarray.jl***
```julia
map(f, A) = collect(Generator(f,A)) # default to returning an Array for `map` on general iterators
map(f, ::AbstractDict) = error("map is not defined on dictionaries")
map(f, ::AbstractSet) = error("map is not defined on sets")
```
***julia\base\generator.jl***
```julia
struct Generator{I,F}
    f::F
    iter::I
end

Generator(f, I1, I2, Is...) = Generator(a->f(a...), zip(I1, I2, Is...))
```

https://docs.juliacn.com/latest/base/collections/#Base.map

In [5]:
map(x -> x * 2, [1, 2, 3])

3-element Vector{Int64}:
 2
 4
 6

### 匿名函数do-block作参数
https://docs.juliacn.com/latest/manual/functions/#Do-Block-Syntax-for-Function-Arguments<br>
匿名函数作为函数参数时，如果行数较多，不好看，可以使用 `do-block`匿名函数

In [9]:
map(x->begin
           if x < 0 && iseven(x)
               return 0
           elseif x == 0
               return 1
           else
               return x
           end
       end,
    [-2, 0, -3, 4, 5])

5-element Vector{Int64}:
  0
  1
 -3
  4
  5

In [12]:
map([-2, 0, -3, 4, 5]) do x  # do要写在map后面,即do前断行的话会报错
    if x < 0 && iseven(x)
        return 0
    elseif x == 0
        return 1
    else
        return x
    end
end

5-element Vector{Int64}:
  0
  1
 -3
  4
  5

`do x` 语法创建一个带有参数 `x` 的匿名函数，并将其作为第一个参数传递给 `map`。 类似地，`do a,b` 将创建一个有两个参数的匿名函数。 请注意，`do (a,b)` 将创建一个单参数匿名函数，其参数是一个要解构的元组。 一个简单的 `do` 会声明接下来是一个形式为 `() -> ..`. 的匿名函数。<br>

这些参数如何初始化取决于「外部」函数；在这里，`map` 将会依次将 `x` 设置为`-2, 0, -3, 4, 5`，再分别调用调用匿名函数，正如在` map(func, [-2, 0, -3, 4, 5])` 语法中所发生的。

***julia\base\io.jl***
```julia
function open(f::Function, args...; kwargs...)
    io = open(args...; kwargs...)
    try
        f(io)
    finally
        close(io)
    end
end
```
可以如下使用：
```julia
open("outfile", "w") do io
    write(io, data)
end
```

在这里，`open` 首先打开要写入的文件，接着将结果输出流传递给你在 `do ... end` 代码快中定义的匿名函数。在你的函数退出后，`open` 将确保流被正确关闭，无论你的函数是正常退出还是抛出了一个异常（`try/finally` 结构会在 流程控制 中描述）。<br>

使用 `do` 代码块语法时，查阅文档或实现有助于了解用户函数的参数是如何初始化的。<br>

类似于其他的内部函数， `do` 代码块也可以“捕获”上一个作用域的变量。例如，上一个 `open...do` 的例子中变量 data 是从外部作用域捕获的。捕获变量可能会给性能优化带来挑战，详见 性能建议。

附：
```julia
help?>open

julia> io = open("myfile.txt", "w");
  
julia> write(io, "Hello world!");

julia> close(io);

julia> io = open("myfile.txt", "r");

julia> read(io, String)
"Hello world!"

julia> write(io, "This file is read only")
ERROR: ArgumentError: write failed, IOStream is not writeable
[...]

julia> close(io)

julia> io = open("myfile.txt", "a");

julia> write(io, "This stream is not read only")
28

julia> close(io)

julia> rm("myfile.txt")
```

In [14]:
sqrt(3,6) do x,y
    x + y
    end

MethodError: MethodError: no method matching sqrt(::var"#21#22", ::Int64, ::Int64)
The function `sqrt` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  sqrt(!Matched::Missing)
   @ Base math.jl:1533
  sqrt(!Matched::ComplexF16)
   @ Base math.jl:1512
  sqrt(!Matched::BigInt)
   @ Base mpfr.jl:703
  ...


In [15]:
function f_multipy(f::Function, r1::Real, r2::Real)
    f_result = f(r1,r2)
    f_result * 2
end

f_multipy (generic function with 1 method)

In [16]:
f_multipy(1,2) do x, y
    x + y
end

6

**`do-block`作函数参数时，函数参数位置只能在第一个位置**，测试如下

In [31]:
function f_multipy2(r1::Real, f::Function, r2::Real)
    f_result = f(r1,r2)
    f_result * 3
end

f_multipy2(1,2) do x, y
    x + y
end

MethodError: MethodError: no method matching f_multipy2(::var"#43#44", ::Int64, ::Int64)
The function `f_multipy2` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  f_multipy2(!Matched::Real, !Matched::Function, ::Real)
   @ Main e:\Projects.jl\Training.jl\7. function methods\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X33sZmlsZQ==.jl:1


In [30]:
function f_multipy3(r1::Real, r2::Real, f::Function)
    f_result = f(r1,r2)
    f_result * 3
end

f_multipy3(1,2) do x, y
    x + y
end

MethodError: MethodError: no method matching f_multipy3(::var"#41#42", ::Int64, ::Int64)
The function `f_multipy3` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  f_multipy3(!Matched::Real, ::Real, !Matched::Function)
   @ Main e:\Projects.jl\Training.jl\7. function methods\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X34sZmlsZQ==.jl:1


https://zhuanlan.zhihu.com/p/19811813
```julia
yonggang li 2015-07-03
如果是map(arg1,f1,arg2,f2,args...)这样的声明呢?

知乎用户6pEVPF 2015-07-03 ​回复
这就没有办法了，现在仅仅能够将第一个且第一个 function 用 do

知乎用户6pEVPF 2015-07-03
一个语法糖，原本像是 map 这样的函数直接写会有
map( 一个很长的匿名函数写在这里很难看, varlist)

现在写成

map ( varlist) do x
匿名函数的行为写在这里就可以了
end

这会好看很多
```